# Imports

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from xgboost import XGBRegressor

from tensorflow.keras.models import Model
import tensorflow.keras.layers as tfl

# Loading Data

In [40]:
df = pd.read_csv("train_set_final.csv")
val = pd.read_csv("validation_set_final.csv")

In [7]:
df.head()

,Unit,Quarter,Month,Quantity,Year,Booking_Month,Booking_Year
0,CSWBU,Q1,AUG,204,2022,1,2016
1,CSWBU,Q1,OCT,564,2022,1,2016
2,CSWBU,Q1,SEP,1476,2022,1,2016
3,CSWBU,Q1,AUG,374,2002,1,2017
4,CSWBU,Q1,OCT,6094,2002,1,2017


In [41]:
len(val)

0

In [8]:
y = df["Quantity"]
X = df.drop("Quantity", axis=1)

val_y = val["Quantity"]
val_X = val.drop("Quantity", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
preprocessor = ColumnTransformer(
    transformers=[
        ("one_hot", OneHotEncoder(), X_train.columns),
    ]
)

In [22]:
model = LinearRegression()
str(model)

'LinearRegression()'

In [23]:
def give_model(model, **kwargs):
    pipe = Pipeline(steps=[("preprocessor", preprocessor),
                            (model.__class__.__name__, model(**kwargs))])
    pipe.fit(X_train, y_train)
    return pipe

In [27]:
def evaluate(model, on="trrain", **kwargs):
    model = give_model(model, **kwargs)
    if on == "train":
        data = (X_train, y_train)
    elif on == "test":
        data = (X_test, y_test)
    elif on == "val":
        data = (val_X, val_y)
    
    y_pred = model.predict(data[0])
    r2 = r2_score(data[1], y_pred)
    rmse = np.sqrt(mean_squared_error(data[1], y_pred))
    print(f"R2 score: {r2}")
    print(f"RMSE: {rmse}")
    return model, r2, rmse

In [28]:
_, r2, rmse = evaluate(LinearRegression, on="test")

R2 score: 0.32603978787673815
RMSE: 14795.249124776588


In [32]:
_, r2, rmse = evaluate(RandomForestRegressor, on="test", n_estimators=1000, max_depth=10, random_state=42)

R2 score: 0.2732254948771399
RMSE: 15364.02464602094


In [36]:
_, r2, rmse = evaluate(XGBRegressor, on="test", n_estimators=1000, max_depth=20, random_state=42)

R2 score: 0.23818479340434462
RMSE: 15730.045245075213


## Deep Learning!!!

In [39]:
len(val_X)

0

In [38]:
X_train_n = preprocessor.fit_transform(X_train)
X_test_n = preprocessor.transform(X_test)
val_X_n = preprocessor.transform(val_X)

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.